In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import json
from scipy.special import softmax

# test on data from HKT
path = '/mnt/qb/work/mlcolab/hzhou52/kt/hkt/ass13/interactions.csv'
log = pd.read_csv(path, sep='\t') 

In [16]:
user_unique = list(log.user_id.unique())
final = []
for user in user_unique:
    user_log = log.loc[log.user_id == user]
    user_log = user_log.sort_values('timestamp')
    final.append([])
    for index, row in user_log.iterrows():
        final[-1].append([row.skill_id, row.correct])

In [3]:
base_path = '/mnt/qb/work/mlcolab/hzhou52/kt/test/graph/ass1213_from_hkt/'
# with open(base_path+'ass13_seq.json', 'w') as f:
#     json.dump(final, f)

f = open(base_path+'ass13_seq.json')
final = json.load(f)


In [18]:
# from https://github.com/bigdata-ustc/EduData/blob/e8d1d4d12869ddd8cee61b9acb23363cb9d173c1/EduData/Task/KnowledgeTracing/graph.py
import fileinput, tqdm
def _correct_transition_count_graph(count_graph, seq):
    pre_c = None
    for se in seq:
        eid = int(se[0])
        r = int(se[1])
        if pre_c is not None:
            if eid != pre_c and r == 1:
                count_graph[pre_c][eid] += 1
            elif r == 1:
                pass
        if r == 1:
            pre_c = eid
        else:
            pre_c = None

def _output_graph(graph, tar):
    ku_num = len(graph)

    _graph = []

    for i in range(ku_num):
        for j in range(ku_num):
            if i != j and not np.isnan(graph[i][j]) and graph[i][j] > 0:
                _graph.append([i, j, graph[i][j]])

    with open(tar, 'w') as wf:
    # with wf_open(tar) as wf:
        json.dump(_graph, wf, indent=2)

def correct_transition_count_graph(ku_num, seqs, tar=None, input_is_file=True):
    count_graph = [[0] * ku_num for _ in range(ku_num)]

    if input_is_file:
        with fileinput.input(files=src) as f:
            for line in tqdm(f, "constructing coorect transition graph"):
                if not line.strip():  # pragma: no cover
                    continue
                seq = json.loads(line)
                _correct_transition_count_graph(count_graph, seq)
    else:
        for seq in seqs:
            _correct_transition_count_graph(count_graph, seq)

    if tar is not None:
        _output_graph(count_graph, tar)

    return count_graph



In [19]:
ku_num = log.skill_id.unique().shape[0]
tar = base_path + 'correct_transition_count_graph.json'
count_graph = correct_transition_count_graph(ku_num, final, tar, input_is_file=False)

In [ ]:
def _row_normalize(count_graph, diagonal_value=None, normalized_by_softmax=False, skip_zero_row=False):
    _graph = np.asarray(count_graph)
    zero_rows_indices = None

    if skip_zero_row:
        zero_rows_indices = np.sum(_graph, axis=-1) == 0

    if normalized_by_softmax:
        if diagonal_value is None:
            _graph = softmax(_graph, axis=-1)
        else:
            _graph = _graph.astype(float)
            np.fill_diagonal(_graph, -np.inf)
            _graph = softmax(_graph, axis=-1)
            if diagonal_value is not None:
                np.fill_diagonal(_graph, diagonal_value)
    else:
        _offset_graph = _graph + 1e-50
        if diagonal_value is not None:
            np.fill_diagonal(_offset_graph, 0.0)
            _graph = (_offset_graph.T / _offset_graph.sum(axis=-1)).T
            _graph[_graph <= 1e-40] = 0.0
            np.fill_diagonal(_graph, diagonal_value)
        else:
            _graph = (_offset_graph.T / _offset_graph.sum(axis=-1)).T
            _graph[_graph <= 1e-40] = 0.0

    if skip_zero_row:
        _graph[zero_rows_indices] = 0.0

    return _graph.tolist()

def correct_transition_graph(ku_num, src, tar=None, input_is_file=True, diagonal_value=0.):
    """
    When a concept is mastered, how much probability is it to be transferred to another concept.
    For example,
    ```
    [[0, 1], [1, 0], [1, 1], [2, 1]]
    [[2, 0], [1, 0], [0, 1], [2, 1]]
    ```
    When concept #0 is mastered (i.e., 1st in seq #1, 3rd in seq #2),
    only concept # 2 can be mastered (4th in seq #2).
    Thus, the transition probabilty for concept #0 is [0, 0, 1],
    which mastering concept #0 can influence mastering concept #2 more thant concept #1.
    """
    count_graph = correct_transition_count_graph(ku_num, src, tar=None, input_is_file=input_is_file)
    _transition_graph = _row_normalize(count_graph, diagonal_value, skip_zero_row=True)

    if tar is not None:
        _output_graph(_transition_graph, tar)

    return _transition_graphs

In [24]:
correct_transition_graph(ku_num, final, tar, input_is_file=False)

In [4]:
# ass13 original file

import fileinput
import json
import warnings
import numpy as np
import pandas as pd
import time
from datetime import datetime

import ipdb

path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment/2012-2013-data-with-predictions-4-final.csv'
data = pd.read_csv(path, encoding = "ISO-8859-15",low_memory=False)
time_format = "%Y-%m-%d %H:%M:%S"

In [5]:
tmp1 = []
tmp2 = []
for index, row in data.iterrows():
    start = row['start_time']
    start = time.mktime(datetime.strptime(start.split('.')[0], time_format).timetuple())
    tmp1.append(start)
    # data['start_time'][index] = start
    end = row['end_time']
    end = time.mktime(datetime.strptime(end.split('.')[0], time_format).timetuple())
    tmp2.append(end)
    # data['end_time'][index] = end
data['start_timestamp'] = tmp1
data['end_timestamp'] = tmp2

In [11]:
# remove 
data.columns

Index(['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id',
       'assistment_id', 'start_time', 'end_time', 'problem_type', 'original',
       'correct', 'bottom_hint', 'hint_count', 'actions', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)',
       'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)', 'start_timestamp', 'end_timestamp'],
      dtype='object')

In [13]:
data[~data['skill_id'].isna()]

,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,...,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED),start_timestamp,end_timestamp
1,138083797,Rounding,365981,61394,573819,204043,2012-10-09 11:01:52,2012-10-09 11:02:13.182,algebra,1,...,NaN,74.29,0,138083797,0.361323,0.0,0.766925,0.000000,1.349773e+09,1.349773e+09
2,142332619,Multiplication and Division Integers,426415,61394,734130,247525,2013-03-07 10:53:20,2013-03-07 10:53:28.661,algebra,1,...,NaN,00,0,142332619,0.361323,0.0,0.766925,0.442968,1.362650e+09,1.362650e+09
3,145939397,Proportion,86686,61394,821352,48081,2013-08-20 19:54:56,2013-08-20 19:55:21.753,algebra,1,...,NaN,3.8,0,145939397,0.775000,0.0,0.766925,0.912281,1.377021e+09,1.377021e+09
5,140218191,Exponents,401234,76592,639711,228996,2012-12-12 21:00:55,2012-12-12 21:01:07.536,algebra,1,...,NaN,1024,0,140218191,0.361323,0.0,0.766925,0.000000,1.355342e+09,1.355342e+09
7,138367797,Equation Solving Two or Fewer Steps,87699,78401,581453,49019,2012-10-16 10:30:54,2012-10-16 10:31:55.445,algebra,1,...,NaN,-4,0,138367797,0.361323,0.0,0.766925,0.000000,1.350376e+09,1.350376e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6123259,146095368,NaN,91034,227693,831346,51710,2013-08-29 13:20:27,2013-08-29 13:20:52.98,algebra,1,...,NaN,5,0,146095368,0.361323,0.0,0.336529,0.762689,1.377775e+09,1.377775e+09
6123260,146101974,Square Root,117,227788,823672,117,2013-08-29 16:18:36,2013-08-29 16:19:08.388,choose_1,1,...,NaN,7 and 8,0,146101974,0.361323,0.0,0.336529,0.442968,1.377786e+09,1.377786e+09
6123262,146134849,Unit Conversion Within a System,123251,227839,772344,64904,2013-08-31 15:15:01,2013-08-31 15:15:08.024,choose_1,0,...,NaN,Ok. I have studied this example and am ready t...,0,146134849,0.361323,0.0,0.766925,0.000000,1.377955e+09,1.377955e+09
6123264,146130573,Multiplication Fractions,86906,227908,517761,48296,2013-08-31 00:23:38,2013-08-31 00:25:16.236,algebra,1,...,NaN,7/24,0,146130573,0.361323,0.0,0.336529,0.369140,1.377901e+09,1.377902e+09


In [10]:
data

,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,...,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED),start_timestamp,end_timestamp
0,137792159,NaN,557460,61394,565736,341511,2012-09-28 15:11:27,2012-09-28 15:11:36.856,choose_1,1,...,NaN,she,0,137792159,0.361323,0.0,0.336529,0.000000,1.348838e+09,1.348838e+09
1,138083797,Rounding,365981,61394,573819,204043,2012-10-09 11:01:52,2012-10-09 11:02:13.182,algebra,1,...,NaN,74.29,0,138083797,0.361323,0.0,0.766925,0.000000,1.349773e+09,1.349773e+09
2,142332619,Multiplication and Division Integers,426415,61394,734130,247525,2013-03-07 10:53:20,2013-03-07 10:53:28.661,algebra,1,...,NaN,00,0,142332619,0.361323,0.0,0.766925,0.442968,1.362650e+09,1.362650e+09
3,145939397,Proportion,86686,61394,821352,48081,2013-08-20 19:54:56,2013-08-20 19:55:21.753,algebra,1,...,NaN,3.8,0,145939397,0.775000,0.0,0.766925,0.912281,1.377021e+09,1.377021e+09
4,137111284,NaN,399669,76592,557216,227869,2012-09-10 17:20:10,2012-09-10 17:24:56.579,choose_1,1,...,NaN,C (wr - 1)(wr + 1),0,137111284,0.361323,0.0,0.766925,0.000000,1.347290e+09,1.347291e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6123265,146114936,NaN,330756,227984,832474,180547,2013-08-30 10:10:55,2013-08-30 10:11:08.935,choose_1,1,...,NaN,yes,0,146114936,0.361323,0.0,0.766925,0.000000,1.377850e+09,1.377850e+09
6123266,146131268,NaN,765964,228009,831417,491711,2013-08-31 10:23:24,2013-08-31 10:23:27.757,choose_1,1,...,NaN,Biotic factors in a forest,0,146131268,0.361323,0.0,0.766925,0.000000,1.377937e+09,1.377937e+09
6123267,146118011,Addition and Subtraction Integers,84891,228039,832875,46603,2013-08-30 11:29:35,2013-08-30 11:30:05.517,algebra,1,...,NaN,-19,0,146118011,0.361323,0.0,0.211268,1.000000,1.377855e+09,1.377855e+09
6123268,146118739,NaN,543973,228055,832844,331778,2013-08-30 11:52:37,2013-08-30 11:52:41.251,choose_1,1,...,NaN,Yes,0,146118739,0.361323,0.0,0.336529,0.442968,1.377856e+09,1.377856e+09


In [ ]:
2711813